In [7]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("data_input/chinook.db")

In [8]:
emp = pd.read_sql_query("select * from employees", conn)
emp

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
5,6,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
6,7,King,Robert,IT Staff,6.0,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
7,8,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [17]:
track = pd.read_sql_query("select * from tracks where TrackId = 5", conn)
track

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [20]:
genre = pd.read_sql_query(
    '''
        SELECT 
            trk.Name as TrackNm, 
            trk.Composer, 
            trk.UnitPrice, 
            genr.Name as GenreNm, 
            genr.GenreId, 
            trk.TrackId 
        FROM tracks AS trk,     
             genres AS genr 
        WHERE trk.GenreId = genr.GenreId 
              AND trk.TrackId = 5 
              AND genr.GenreId = 1
     ''', conn)
genre.head()

,TrackNm,Composer,UnitPrice,GenreNm,GenreId,TrackId
0,Princess of the Dawn,Deaffy & R.A. Smith-Diesel,0.99,Rock,1,5


In [22]:
pd.read_sql_query("SELECT * FROM media_types", conn)

,MediaTypeId,Name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


In [22]:
ttl_sales = pd.read_sql_query(
    '''
    SELECT 
         inv.BillingCountry AS Country, 
         genr.Name AS Genre, 
         inv.InvoiceDate, 
         SUM(invs.Total) AS Total, 
         AVG(invs.Total) AS avg_inv
    FROM invoices AS invs, 
         invoice_items AS inv_itm, 
         tracks AS trk, 
         genres AS genr, 
         media_types AS med_typ 
    WHERE    invs.InvoiceId = inv_itm.InvoiceId 
         AND inv_itm.TrackId = trk.TrackId 
         AND trk.GenreId = genr.GenreId 
         AND trk.MediaTypeId = med_typ.MediaTypeId 
         AND med_typ.MediaTypeId = 1 
    GROUP BY Genre 
    ORDER BY Total
    '''
    ,conn, parse_dates='InvoiceDate')

ttl_sales['Month'] = ttl_sales["InvoiceDate"].dt.month_name()
ttl_sales

,Country,Genre,InvoiceDate,Total,avg_inv,Month
0,Brazil,Pop,2012-12-07,83.16,13.860000,December
1,USA,Rock And Roll,2012-11-06,83.16,13.860000,November
2,Canada,Bossa Nova,2013-01-30,86.13,5.742000,January
3,USA,Easy Listening,2011-12-01,138.60,13.860000,December
4,Czech Republic,Electronica/Dance,2009-07-11,149.62,12.468333,July
5,USA,Heavy Metal,2010-09-23,161.37,13.447500,September
6,Portugal,World,2012-02-01,166.32,13.860000,February
7,Canada,Hip Hop/Rap,2012-05-04,166.41,9.788824,May
8,Brazil,Soundtrack,2012-12-07,242.55,12.127500,December
9,India,R&B/Soul,2009-07-08,272.25,9.387931,July


In [11]:
ttl_sales['month_year'] = pd.to_datetime(ttl_sales['InvoiceDate']).dt.to_period('M')
ttl_sales

,Country,Genre,InvoiceDate,Total,avg_inv,Month,month_year
0,Brazil,Pop,2012-12-07,83.16,13.860000,December,2012-12
1,USA,Rock And Roll,2012-11-06,83.16,13.860000,November,2012-11
2,Canada,Bossa Nova,2013-01-30,86.13,5.742000,January,2013-01
3,USA,Easy Listening,2011-12-01,138.60,13.860000,December,2011-12
4,Czech Republic,Electronica/Dance,2009-07-11,149.62,12.468333,July,2009-07
5,USA,Heavy Metal,2010-09-23,161.37,13.447500,September,2010-09
6,Portugal,World,2012-02-01,166.32,13.860000,February,2012-02
7,Canada,Hip Hop/Rap,2012-05-04,166.41,9.788824,May,2012-05
8,Brazil,Soundtrack,2012-12-07,242.55,12.127500,December,2012-12
9,India,R&B/Soul,2009-07-08,272.25,9.387931,July,2009-07


In [64]:
ttl_sales['month_year'] = ttl_sales['InvoiceDate'].dt.strftime('%m-%Y')
ttl_sales

,Country,Genre,InvoiceDate,Total,Month,month_year
0,Chile,Science Fiction,2010-01-13,102.41,January,01-2010
1,USA,Comedy,2012-09-27,112.30,September,09-2012
2,USA,Sci Fi & Fantasy,2012-08-05,198.87,August,08-2012
3,Czech Republic,Drama,2013-11-13,544.61,November,11-2013
4,Sweden,TV Shows,2010-01-10,817.71,January,01-2010


In [65]:
country_sales = pd.read_sql_query(
    '''
    SELECT invs.BillingCountry AS Country, 
           genr.Name AS Genre, 
           med_typ.Name AS Media_Nm, 
           sum(invs.Total) AS Total
    FROM invoices AS invs, 
         invoice_items AS invs_itm, 
         tracks AS trks, 
         genres AS genr, 
         media_types AS med_typ 
    WHERE 
         invs.InvoiceId = invs_itm.InvoiceId 
     AND invs_itm.TrackId = trks.TrackId 
     AND trks.GenreId = genr.GenreId 
     AND trks.MediaTypeId = med_typ.MediaTypeId 
     AND invs.BillingCountry = 'USA'
     GROUP BY Genre 
     ORDER BY Total DESC
    '''
    , conn)
country_sales

,Country,Genre,Media_Nm,Total
0,USA,Rock,MPEG audio file,1526.14
1,USA,Latin,MPEG audio file,754.67
2,USA,Metal,MPEG audio file,554.45
3,USA,Alternative & Punk,MPEG audio file,415.92
4,USA,Jazz,MPEG audio file,202.95
5,USA,TV Shows,Protected MPEG-4 video file,191.70
6,USA,Drama,Protected MPEG-4 video file,143.16
7,USA,Blues,MPEG audio file,126.72
8,USA,Comedy,Protected MPEG-4 video file,90.44
9,USA,Classical,Protected AAC audio file,87.20


In [25]:
country_sales['Media_Nm'] = country_sales['Media_Nm'].astype('category', errors='raise')
country_sales.dtypes

Country       object
Genre         object
Media_Nm    category
Total        float64
dtype: object

In [26]:
country_sales.head()

,Country,Genre,Media_Nm,Total
0,USA,Rock,MPEG audio file,1526.14
1,USA,Latin,MPEG audio file,754.67
2,USA,Metal,MPEG audio file,554.45
3,USA,Alternative & Punk,MPEG audio file,415.92
4,USA,Jazz,MPEG audio file,202.95


In [64]:
ttl_sales_media = pd.read_sql_query(
    '''
        SELECT invs.BillingCountry AS Country, genr.Name AS Genre, med_typ.Name, SUM(invs.Total) AS Total 
        FROM invoices AS invs, invoice_items AS invs_itm, tracks AS trks, genres AS genr, media_types AS med_typ 
        WHERE     invs.InvoiceId = invs_itm.InvoiceId 
              AND invs_itm.TrackId = trks.TrackId 
              AND trks.GenreId = genr.GenreId 
              AND trks.MediaTypeId = med_typ.MediaTypeId 
              AND med_typ.MediaTypeId = 1 
        GROUP BY Genre 
        ORDER BY Total
    '''
    , conn)
ttl_sales_media

,Country,Genre,Name,Total
0,Brazil,Pop,MPEG audio file,83.16
1,USA,Rock And Roll,MPEG audio file,83.16
2,Canada,Bossa Nova,MPEG audio file,86.13
3,USA,Easy Listening,MPEG audio file,138.60
4,Czech Republic,Electronica/Dance,MPEG audio file,149.62
5,USA,Heavy Metal,MPEG audio file,161.37
6,Portugal,World,MPEG audio file,166.32
7,Canada,Hip Hop/Rap,MPEG audio file,166.41
8,Brazil,Soundtrack,MPEG audio file,242.55
9,India,R&B/Soul,MPEG audio file,272.25


In [33]:
album = pd.read_sql_query(
    '''
    SELECT * FROM albums
    '''
    , conn)
album.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


In [66]:
total_sls = pd.read_sql_query(
    '''
    SELECT 
        albm.Title AS Album, 
        art.Name AS Artist, 
        sum(inv.Total) AS TotalSales
    FROM
        invoices AS inv, 
        invoice_items AS inv_itm, 
        tracks AS trk, 
        albums AS albm, 
        artists AS art 
     WHERE 
            inv.InvoiceId = inv_itm.InvoiceId 
        AND inv_itm.TrackId = trk.TrackId 
        AND trk.AlbumId = albm.AlbumId 
        AND albm.ArtistId = art.ArtistId 
     GROUP BY albm.Title
     ORDER BY TotalSales DESC
     '''
     , conn)
total_sls.head(10)

,Album,Artist,TotalSales
0,Greatest Hits,Lenny Kravitz,372.51
1,"Lost, Season 2",Lost,290.18
2,"Heroes, Season 1",Heroes,238.61
3,"Lost, Season 1",Lost,223.65
4,"Lost, Season 3",Lost,211.80
5,"Battlestar Galactica, Season 3",Battlestar Galactica,202.80
6,Minha Historia,Chico Buarque,185.13
7,"The Office, Season 3",The Office,170.93
8,Ao Vivo [IMPORT],Zeca Pagodinho,161.74
9,"Battlestar Galactica (Classic), Season 1",Battlestar Galactica (Classic),157.10


In [ ]:
ttl_sales = pd.read_sql_query("")
ttl_sales.head()

In [67]:
emp_sales = pd.read_sql_query(
    '''
    SELECT 
            (empl.FirstName||' '||empl.LastName) AS FullName, 
            sum(invs.Total) AS TotalSales,
            invs.InvoiceDate AS Period 
    FROM 
            employees AS empl, 
            customers AS cust, 
            invoices AS invs 
    WHERE 
                empl.EmployeeId = cust.SupportRepId 
            AND cust.CustomerId = invs.CustomerId 
            AND empl.Title = 'Sales Support Agent' 
    GROUP BY FullName, invs.InvoiceDate
    '''
    , conn, parse_dates='Period')

emp_sales['Period'] = pd.to_datetime(emp_sales['Period']).dt.to_period('M')                         
emp_sales

,FullName,TotalSales,Period
0,Jane Peacock,0.99,2009-01
1,Jane Peacock,1.98,2009-02
2,Jane Peacock,3.96,2009-02
3,Jane Peacock,5.94,2009-02
4,Jane Peacock,8.91,2009-02
...,...,...,...
398,Steve Johnson,0.99,2013-10
399,Steve Johnson,5.94,2013-11
400,Steve Johnson,25.86,2013-11
401,Steve Johnson,1.98,2013-12


In [56]:
empl_sales_nm = empl_sales.pivot_table(
    index= "Period",
    values= "TotalSales",
    aggfunc='sum'
).sort_values(by="Period", ascending=True)

#emp_sales_nm.head()
empl_sales_nm['TotalSales'].fillna(0).head()


Period
2009-01    35.64
2009-02    37.62
2009-03    37.62
2009-04    37.62
2009-05    37.62
Freq: M, Name: TotalSales, dtype: float64

In [69]:
emp_total_sales = pd.crosstab(
    index=emp_sales['Period'], 
    columns=emp_sales['FullName'],
    values=emp_sales['TotalSales'],
    aggfunc = 'sum')

emp_total_sales.head()

FullName,Jane Peacock,Margaret Park,Steve Johnson
Period,,,
2009-01,0.99,23.76,10.89
2009-02,20.79,2.97,13.86
2009-03,1.98,13.86,21.78
2009-04,18.81,16.83,1.98
2009-05,10.89,1.98,24.75


In [52]:
emp_total_sales.melt

<bound method DataFrame.melt of FullName  Jane Peacock  Margaret Park  Steve Johnson
Period                                              
2009-01           0.99          23.76          10.89
2009-02          20.79           2.97          13.86
2009-03           1.98          13.86          21.78
2009-04          18.81          16.83           1.98
2009-05          10.89           1.98          24.75
2009-06           1.98          10.89          24.75
2009-07          22.77           3.96          10.89
2009-08          30.69           6.93            NaN
2009-09           0.99          28.71           7.92
2009-10            NaN           7.92          29.70
2009-11           3.96          31.68           1.98
2009-12           9.90          11.88          15.84
2010-01           3.96            NaN          48.66
2010-02          31.77           5.94           8.91
2010-03          34.72           9.90            NaN
2010-04          26.73           1.98           8.91
2010-05       

In [60]:
dt_sales = pd.read_sql_query(
    '''
    SELECT 
        (empl.FirstName||' '||empl.LastName) AS FullName, 
        sum(invs.Total) AS TotalSales, 
        invs.BillingCountry AS Country 
    FROM 
        employees AS empl, 
        customers AS cust, 
        invoices AS invs 
    WHERE 
            empl.EmployeeId = +cust.SupportRepId 
        AND cust.CustomerId = +invs.CustomerId 
        AND empl.Title = 'Sales Support Agent' 
    GROUP BY BillingCountry 
    ORDER BY FullName, TotalSales
    '''
    , conn)

dt_sales.head()

,FullName,TotalSales,Country
0,Jane Peacock,41.62,Finland
1,Jane Peacock,45.62,Hungary
2,Jane Peacock,45.62,Ireland
3,Jane Peacock,75.26,India
4,Jane Peacock,112.86,United Kingdom


In [61]:
dt_sales.stack().head(20)

0  FullName        Jane Peacock
   TotalSales             41.62
   Country              Finland
1  FullName        Jane Peacock
   TotalSales             45.62
   Country              Hungary
2  FullName        Jane Peacock
   TotalSales             45.62
   Country              Ireland
3  FullName        Jane Peacock
   TotalSales             75.26
   Country                India
4  FullName        Jane Peacock
   TotalSales            112.86
   Country       United Kingdom
5  FullName        Jane Peacock
   TotalSales            156.48
   Country              Germany
6  FullName        Jane Peacock
   TotalSales             190.1
dtype: object